In [1]:
import ROOT



Welcome to JupyROOT 6.30/02


In [12]:
def process_file(file_path, tree_name):
    # Open the ROOT file in read mode
    file_1 = ROOT.TFile(file_path, "READ")
    tree = file_1.Get(tree_name)

    # Arrays to store pt values and weights
    lead_pts = []
    sublead_pts = []
    weights = []
    weight_central_values = []

    if not tree:
        print(f"Tree not found in file: {file_path}")
    else:
        for event in tree:
            if not hasattr(event, 'lead_pt') or not hasattr(event, 'sublead_pt'):
                continue

            lead_pt = event.lead_pt
            sublead_pt = event.sublead_pt
            lead_mvaID_WP90 = event.lead_mvaID_WP90
            sublead_mvaID_WP90 = event.sublead_mvaID_WP90
            weight = getattr(event, 'weight', 1)  # Use 1 if weight is not available
            weight_central = getattr(event, 'weight_central', 1)  # Use 1 if weight_central is not available
            
            if lead_mvaID_WP90 != 1 or sublead_mvaID_WP90 != 1:
                continue

            lead_pts.append(lead_pt)
            sublead_pts.append(sublead_pt)
            weights.append(weight)
            weight_central_values.append(weight_central)

    # Close the ROOT file
    file_1.Close()

    # Return the pt values and weights
    return lead_pts, sublead_pts, weights, weight_central_values


In [13]:
# Data file paths
data_file_paths = [
    "../../output_root/Data_EraE.root",
    "../../output_root/Data_EraF.root",
    "../../output_root/Data_EraG.root"
]

# Process background files
background_files = [
    ("../../output_root/GGJets.root", "GGJets"),
    ("../../output_root/GJetPt20To40.root", "GJetPt20To40"),
    ("../../output_root/GJetPt40.root", "GJetPt40"),
    ("../../output_root/GluGluHToGG.root", "GluGluHToGG"),
    ("../../output_root/ttHToGG.root", "ttHToGG"),
    ("../../output_root/VBFHToGG.root", "VBFHToGG"),
    ("../../output_root/VHToGG.root", "VHToGG")
]

# Define the signal file path
signal_file = "../../output_root/GluGluToHH.root"




# Tree and variable names
tree_name = "DiphotonTree/data_125_13TeV_NOTAG"

# Branching ratio
BR_HToGG = 2.270E-03
BR_HTobb = 5.824E-01
BR_HTogg =  2.270E-03   #https://twiki.cern.ch/twiki/bin/view/LHCPhysics/CERNYellowReportPageBR

# Cross sections for backgrounds and signal
cross_sections = {
    "GGJets": 88.75,
    "GJetPt20To40": 242.5,
    "GJetPt40": 919.1,
    "GluGluHToGG": 52.23 * BR_HToGG,  
    "ttHToGG": 0.0013  ,
    "VBFHToGG": 0.00926 ,
    "VHToGG": 0.00545 ,
    "GluGluToHH":  34.43 * BR_HTobb * BR_HTogg * 2   # 0.1186     #0.1186   # ~34fb*BR(H-bb)*BR(H-gg)*2  = ~ 0.26% Checked
}



integrated_luminosities = {
    "Data_EraE": 5.8070,
    "Data_EraF": 17.7819,
    "Data_EraG": 3.0828
}

In [14]:
# Total integrated luminosity
total_integrated_luminosity = sum(integrated_luminosities.values())
print(f"Total integrated luminosity: {total_integrated_luminosity} fb")


Total integrated luminosity: 26.6717 fb


In [15]:
# Process data files
data_lead_pts = []
data_sublead_pts = []
for data_file_path in data_file_paths:
    lead_pts, sublead_pts, _, _ = process_file(data_file_path, tree_name)
    data_lead_pts.extend(lead_pts)
    data_sublead_pts.extend(sublead_pts)

In [45]:
# Process background files and create histograms
background_hists_lead = {}
background_hists_sublead = {}
hex_colors = ["#3f90da", "#ffa90e", "#bd1f01", "#94a4a2", "#832db6", "#a96b59", "#e76300", "#b9ac70", "#717581", "#92dadd"]
rgb_colors = [(int(h[1:3], 16), int(h[3:5], 16), int(h[5:], 16)) for h in hex_colors]

for i, (background_file, bg_name) in enumerate(background_files):
    bg_lead_pts, bg_sublead_pts, bg_weights, bg_weight_central = process_file(background_file, tree_name)
    hist_lead = ROOT.TH1F(f"hist_lead_{bg_name}", f"{bg_name} Leading Photon Pt", 30, 150, 200)
    hist_sublead = ROOT.TH1F(f"hist_sublead_{bg_name}", f"{bg_name} Subleading Photon Pt", 30, 150, 200)
    cross_section = cross_sections[bg_name]
    for lead_pt, sublead_pt, weight, weight_central in zip(bg_lead_pts, bg_sublead_pts, bg_weights, bg_weight_central):
        adjusted_weight = (weight * 1000 * total_integrated_luminosity * cross_section) / weight_central
        hist_lead.Fill(lead_pt, adjusted_weight)
        hist_sublead.Fill(sublead_pt, adjusted_weight)
    r, g, b = rgb_colors[i]
    hist_lead.SetFillColor(ROOT.TColor.GetColor(r, g, b))
    hist_sublead.SetFillColor(ROOT.TColor.GetColor(r, g, b))
    background_hists_lead[bg_name] = hist_lead
    background_hists_sublead[bg_name] = hist_sublead

Warning in <TROOT::Append>: Replacing existing TH1: hist_lead_VHToGG (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist_sublead_VHToGG (Potential memory leak).


In [46]:
# Process signal file and create histograms
signal_lead_pts, signal_sublead_pts, signal_weights, signal_weight_central = process_file(signal_file, tree_name)
signal_hist_lead = ROOT.TH1F("hist_lead_signal", "Signal Leading Photon Pt", 30, 150, 200)
signal_hist_sublead = ROOT.TH1F("hist_sublead_signal", "Signal Subleading Photon Pt", 30, 150, 200)
signal_cross_section = cross_sections["GluGluToHH"]

for lead_pt, sublead_pt, weight, weight_central in zip(signal_lead_pts, signal_sublead_pts, signal_weights, signal_weight_central):
    adjusted_weight = (weight * 1000 * total_integrated_luminosity * signal_cross_section) / weight_central
    signal_hist_lead.Fill(lead_pt, adjusted_weight)
    signal_hist_sublead.Fill(sublead_pt, adjusted_weight)

Warning in <TROOT::Append>: Replacing existing TH1: hist_lead_signal (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist_sublead_signal (Potential memory leak).


In [47]:
signal_hist_lead.SetLineColor(ROOT.kGreen)
signal_hist_lead.SetLineWidth(2)
signal_hist_sublead.SetLineColor(ROOT.kGreen)
signal_hist_sublead.SetLineWidth(2)

signal_hist_lead.Scale(20)
signal_hist_sublead.Scale(20)

In [41]:
# # Create canvases
# canvas_lead = ROOT.TCanvas("canvas_lead", "Leading Photon Pt", 800, 600)
# canvas_sublead = ROOT.TCanvas("canvas_sublead", "Subleading Photon Pt", 800, 600)

# # Draw lead pt plot
# canvas_lead.cd()
# stack_lead = ROOT.THStack("stack_lead", "")
# for bg_name, hist in background_hists_lead.items():
#     stack_lead.Add(hist)
# stack_lead.Draw("HIST")
# stack_lead.GetXaxis().SetTitle("Leading Photon Pt [GeV]")
# stack_lead.GetYaxis().SetTitle("Events")
# stack_lead.GetYaxis().SetTitleOffset(1.4)
# stack_lead.GetYaxis().SetLabelSize(0.03)
# # stack_lead.GetYaxis().SetTitleOffset(0.30)

# max_y_lead = max(stack_lead.GetMaximum(), signal_hist_lead.GetMaximum())
# stack_lead.SetMaximum(max_y_lead * 1.5)
# signal_hist_lead.Draw("SAME HIST")

# # Create data histogram for lead pt
# hist_data_lead = ROOT.TH1F("hist_data_lead", "", 30, 0, 300)
# for lead_pt in data_lead_pts:
#     hist_data_lead.Fill(lead_pt)

# hist_data_lead.SetMarkerStyle(20)
# hist_data_lead.SetMarkerSize(1.2)
# hist_data_lead.SetMarkerColor(ROOT.kBlack)
# hist_data_lead.Draw("SAME E1")

# # Add legend for lead pt
# legend_lead = ROOT.TLegend(0.5, 0.5, 0.8, 0.8)
# legend_lead.AddEntry(hist_data_lead, "Data", "lep")
# for bg_name in background_hists_lead:
#     legend_lead.AddEntry(background_hists_lead[bg_name], bg_name, "f")
# legend_lead.AddEntry(signal_hist_lead, "Signal(GluGluToHH)x10", "l")
# legend_lead.Draw()

# # Draw sublead pt plot
# canvas_sublead.cd()
# stack_sublead = ROOT.THStack("stack_sublead", "")
# for bg_name, hist in background_hists_sublead.items():
#     stack_sublead.Add(hist)
# stack_sublead.Draw("HIST")
# stack_sublead.GetXaxis().SetTitle("Subleading Photon Pt [GeV]")
# stack_sublead.GetYaxis().SetTitle("Events")

# stack_sublead.GetYaxis().SetTitleOffset(1.4)
# stack_sublead.GetYaxis().SetLabelSize(0.03)

# max_y_sublead = max(stack_sublead.GetMaximum(), signal_hist_sublead.GetMaximum())
# stack_sublead.SetMaximum(max_y_sublead * 1.5)
# signal_hist_sublead.Draw("SAME HIST")

# # Create data histogram for sublead pt
# hist_data_sublead = ROOT.TH1F("hist_data_sublead", "", 30, 0, 300)
# for sublead_pt in data_sublead_pts:
#     hist_data_sublead.Fill(sublead_pt)

# hist_data_sublead.SetMarkerStyle(20)
# hist_data_sublead.SetMarkerSize(1.2)
# hist_data_sublead.SetMarkerColor(ROOT.kBlack)
# hist_data_sublead.Draw("SAME E1")

# # Add legend for sublead pt
# legend_sublead = ROOT.TLegend(0.5, 0.5, 0.8, 0.8)
# legend_sublead.AddEntry(hist_data_sublead, "Data", "lep")
# for bg_name in background_hists_sublead:
#     legend_sublead.AddEntry(background_hists_sublead[bg_name], bg_name, "f")
# legend_sublead.AddEntry(signal_hist_sublead, "Signal(GluGluToHH)x10", "l")
# legend_sublead.Draw()

# # Save the canvases as PDFs
# canvas_lead.SaveAs("/afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Leading_Photon_Pt_with_Background_and_Signal.pdf")
# canvas_sublead.SaveAs("/afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Subleading_Photon_Pt_with_Background_and_Signal.pdf")

In [35]:
# #Create canvases
# canvas_lead = ROOT.TCanvas("canvas_lead", "Leading Photon Pt", 800, 600)
# canvas_sublead = ROOT.TCanvas("canvas_sublead", "Subleading Photon Pt", 800, 600)

# # Draw lead pt plot
# canvas_lead.cd()
# stack_lead = ROOT.THStack("stack_lead", "")

# #Create data histogram for lead pt and fill it first
# hist_data_lead = ROOT.TH1F("hist_data_lead", "", 30, 0, 300)
# for lead_pt in data_lead_pts:
#     hist_data_lead.Fill(lead_pt)

# hist_data_lead.SetMarkerStyle(20)
# hist_data_lead.SetMarkerSize(1.2)
# hist_data_lead.SetMarkerColor(ROOT.kBlack)
# hist_data_lead.Draw("E1")

# for bg_name, hist in background_hists_lead.items():
#     stack_lead.Add(hist)
# stack_lead.Draw("SAME HIST")
# stack_lead.GetXaxis().SetTitle("Leading Photon P_{T} [GeV]")
# stack_lead.GetYaxis().SetTitle("Events")
# stack_lead.GetYaxis().SetTitleOffset(1.0)
# stack_lead.GetYaxis().SetLabelSize(0.03)

# # Set ticks on all sides with smaller size
# stack_lead.GetXaxis().SetTickSize(0.02)
# stack_lead.GetYaxis().SetTickSize(0.02)

# stack_lead.SetMaximum(50000) # Set y-axis maximum to 50,000
# signal_hist_lead.Draw("SAME HIST")

# # Add legend for lead pt
# legend_lead = ROOT.TLegend(0.45, 0.55, 0.87, 0.85)
# legend_lead.SetNColumns(2)  # Set number of columns for better layout
# legend_lead.SetBorderSize(1)  # Set border size of the legend box
# legend_lead.SetFillStyle(0)  # Transparent background for the legend box
# legend_lead.SetTextSize(0.02)  #Text size
# legend_lead.AddEntry(hist_data_lead, "Data", "lep")
# legend_lead.AddEntry(background_hists_lead["GGJets"], "#gamma#gamma + Jets", "f")
# legend_lead.AddEntry(background_hists_lead["GJetPt20To40"], "#gamma + Jets (20 < p_{T} < 40)", "f")
# legend_lead.AddEntry(background_hists_lead["GJetPt40"], "#gamma + Jets (p_{T} > 40)", "f")
# legend_lead.AddEntry(background_hists_lead["GluGluHToGG"], "(GluGluHToGG)Higgs #rightarrow #gamma#gamma", "f")
# legend_lead.AddEntry(background_hists_lead["ttHToGG"], "(ttHToGG)Higgs #rightarrow #gamma#gamma", "f")
# legend_lead.AddEntry(background_hists_lead["VBFHToGG"], "(VBFHToGG)Higgs #rightarrow #gamma#gamma", "f")
# legend_lead.AddEntry(background_hists_lead["VHToGG"], "(VHToGG)Higgs #rightarrow #gamma#gamma", "f")
# legend_lead.AddEntry(signal_hist_lead, "Signal(GluGluToHH)x10", "l")
# legend_lead.Draw()

# # Add CMS text for lead pt
# cms_label_lead = ROOT.TLatex()
# cms_label_lead.SetNDC()
# cms_label_lead.SetTextFont(61)
# cms_label_lead.SetTextSize(0.04)
# cms_label_lead.DrawLatex(0.1, 0.91, "CMS")

# # Add "Work in Progress" text for lead pt
# work_label_lead = ROOT.TLatex()
# work_label_lead.SetNDC()
# work_label_lead.SetTextFont(52)
# work_label_lead.SetTextSize(0.03)
# work_label_lead.DrawLatex(0.17, 0.91, "Work in Progress")

# # Add energy information for lead pt
# energy_label_lead = ROOT.TLatex()
# energy_label_lead.SetNDC()
# energy_label_lead.SetTextFont(42)
# energy_label_lead.SetTextSize(0.03)
# energy_label_lead.DrawLatex(0.81, 0.91, "(13.6 TeV)")

# # Save the canvas as PDF
# canvas_lead.SaveAs("/afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Leading_Photon_Pt_with_Background_and_Signal.pdf")

# # Draw sublead pt plot
# canvas_sublead.cd()
# stack_sublead = ROOT.THStack("stack_sublead", "")

# # Create data histogram for sublead pt and fill it first
# hist_data_sublead = ROOT.TH1F("hist_data_sublead", "", 30, 0, 300)
# for sublead_pt in data_sublead_pts:
#     hist_data_sublead.Fill(sublead_pt)

# hist_data_sublead.SetMarkerStyle(20)
# hist_data_sublead.SetMarkerSize(1.2)
# hist_data_sublead.SetMarkerColor(ROOT.kBlack)
# hist_data_sublead.Draw("E1")

# for bg_name, hist in background_hists_sublead.items():
#     stack_sublead.Add(hist)
# stack_sublead.Draw("SAME HIST")
# stack_sublead.GetXaxis().SetTitle("Subleading Photon P_{T} [GeV]")
# stack_sublead.GetYaxis().SetTitle("Events")
# stack_sublead.GetYaxis().SetTitleOffset(1.4)
# stack_sublead.GetYaxis().SetLabelSize(0.03)

# # Set ticks on all sides with smaller size
# stack_sublead.GetXaxis().SetTickSize(0.02)
# stack_sublead.GetYaxis().SetTickSize(0.02)

# stack_sublead.SetMaximum(50000) # Set y-axis maximum to 50,000
# signal_hist_sublead.Draw("SAME HIST")

# # Add legend for sublead pt
# legend_sublead = ROOT.TLegend(0.45, 0.55, 0.87, 0.85)
# legend_sublead.SetNColumns(2)  # Set number of columns for better layout
# legend_sublead.SetBorderSize(1)  # Set border size of the legend box
# legend_sublead.SetFillStyle(0)  #Transparent background for the legend box
# legend_sublead.SetTextSize(0.02)  # Text size
# legend_sublead.AddEntry(hist_data_sublead, "Data", "lep")
# legend_sublead.AddEntry(background_hists_lead["GGJets"], "#gamma#gamma + Jets", "f")
# legend_sublead.AddEntry(background_hists_lead["GJetPt20To40"], "#gamma + Jets (20 < p_{T} < 40)", "f")
# legend_sublead.AddEntry(background_hists_lead["GJetPt40"], "#gamma + Jets (p_{T} > 40)", "f")
# legend_sublead.AddEntry(background_hists_lead["GluGluHToGG"], "(GluGluHToGG)Higgs #rightarrow #gamma#gamma", "f")
# legend_sublead.AddEntry(background_hists_lead["ttHToGG"], "(ttHToGG)Higgs #rightarrow #gamma#gamma", "f")
# legend_sublead.AddEntry(background_hists_lead["VBFHToGG"], "(VBFHToGG)Higgs #rightarrow #gamma#gamma", "f")
# legend_sublead.AddEntry(background_hists_lead["VHToGG"], "(VHToGG)Higgs #rightarrow #gamma#gamma", "f")
# legend_sublead.AddEntry(signal_hist_lead, "Signal(GluGluToHH)x10", "l")
# legend_sublead.Draw()

# # Add CMS text for sublead pt
# cms_label_sublead = ROOT.TLatex()
# cms_label_sublead.SetNDC()
# cms_label_sublead.SetTextFont(61)
# cms_label_sublead.SetTextSize(0.04)
# cms_label_sublead.DrawLatex(0.1, 0.91, "CMS")

# # Add "Work in Progress" text for sublead pt
# work_label_sublead = ROOT.TLatex()
# work_label_sublead.SetNDC()
# work_label_sublead.SetTextFont(52)
# work_label_sublead.SetTextSize(0.03)
# work_label_sublead.DrawLatex(0.17, 0.91, "Work in Progress")

# # Add energy information for sublead pt
# energy_label_sublead = ROOT.TLatex()
# energy_label_sublead.SetNDC()
# energy_label_sublead.SetTextFont(42)
# energy_label_sublead.SetTextSize(0.03)
# energy_label_sublead.DrawLatex(0.81, 0.91, "(13.6 TeV)")

# # Save the canvas as PDF
# canvas_sublead.SaveAs("/afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Subleading_Photon_Pt_with_Background_and_Signal.pdf")


# Ratio plot

In [22]:
# Create canvases
canvas_lead = ROOT.TCanvas("canvas_lead", "Leading Photon Pt", 800, 800)
canvas_lead.SetTicks(1, 1)

# Create pad for main plot for leading photon
pad_main_lead = ROOT.TPad("pad_main_lead", "Main Plot", 0.0, 0.3, 1.0, 1.0)
pad_main_lead.SetTicks(1, 1)
pad_main_lead.SetBottomMargin(0.02)  # Reduce bottom margin to hide X-axis numbers
pad_main_lead.Draw()
pad_main_lead.cd()

# Draw lead pt plot
stack_lead = ROOT.THStack("stack_lead", "")

# Create data histogram for lead pt and fill it first
hist_data_lead = ROOT.TH1F("hist_data_lead", "", 30, 0, 300)
for lead_pt in data_lead_pts:
    hist_data_lead.Fill(lead_pt)

hist_data_lead.SetMarkerStyle(20)
hist_data_lead.SetMarkerSize(1.2)
hist_data_lead.SetMarkerColor(ROOT.kBlack)
hist_data_lead.SetStats(0)  # Remove statistics box
hist_data_lead.Draw("E1")

for bg_name, hist in background_hists_lead.items():
    hist.SetStats(0)  # Remove statistics box
    stack_lead.Add(hist)
stack_lead.Draw("SAME HIST")

# Set y-axis title and adjust size
stack_lead.GetYaxis().SetTitle("Events")
stack_lead.GetYaxis().SetTitleOffset(1.4)
stack_lead.GetYaxis().SetTitleSize(0.05)  # Increase title size
stack_lead.GetYaxis().SetLabelSize(0.03)

# Set ticks on all sides with smaller size
stack_lead.GetXaxis().SetTickSize(0.02)
stack_lead.GetYaxis().SetTickSize(0.02)

stack_lead.SetMaximum(50000)  # Set y-axis maximum to 50,000
signal_hist_lead.SetStats(0)  # Remove statistics box
signal_hist_lead.Draw("SAME HIST")

# Add legend for lead pt
legend_lead = ROOT.TLegend(0.45, 0.55, 0.87, 0.85)
legend_lead.SetNColumns(2)  # Set number of columns for better layout
legend_lead.SetBorderSize(1)  # Set border size of the legend box
legend_lead.SetFillStyle(0)  # Transparent background for the legend box
legend_lead.SetTextSize(0.02)  # Text size
legend_lead.AddEntry(hist_data_lead, "Data", "lep")
legend_lead.AddEntry(background_hists_lead["GGJets"], "#gamma#gamma + Jets", "f")
legend_lead.AddEntry(background_hists_lead["GJetPt20To40"], "#gamma + Jets (20 < p_{T} < 40)", "f")
legend_lead.AddEntry(background_hists_lead["GJetPt40"], "#gamma + Jets (p_{T} > 40)", "f")
legend_lead.AddEntry(background_hists_lead["GluGluHToGG"], "(GluGluHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(background_hists_lead["ttHToGG"], "(ttHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(background_hists_lead["VBFHToGG"], "(VBFHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(background_hists_lead["VHToGG"], "(VHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(signal_hist_lead, "Signal(GluGluToHH)x10", "l")
legend_lead.Draw()

# Add CMS text for lead pt
cms_label_lead = ROOT.TLatex()
cms_label_lead.SetNDC()
cms_label_lead.SetTextFont(61)
cms_label_lead.SetTextSize(0.04)
cms_label_lead.DrawLatex(0.1, 0.91, "CMS")

# Add "Work in Progress" text for lead pt
work_label_lead = ROOT.TLatex()
work_label_lead.SetNDC()
work_label_lead.SetTextFont(52)
work_label_lead.SetTextSize(0.03)
work_label_lead.DrawLatex(0.17, 0.91, "Work in Progress")

# Add energy information for lead pt
energy_label_lead = ROOT.TLatex()
energy_label_lead.SetNDC()
energy_label_lead.SetTextFont(42)
energy_label_lead.SetTextSize(0.03)
energy_label_lead.DrawLatex(0.81, 0.91, "(13.6 TeV)")

# Create pad for ratio plot for leading photon
canvas_lead.cd()
pad_ratio_lead = ROOT.TPad("pad_ratio_lead", "Ratio Plot", 0.0, 0.0, 1.0, 0.3)
pad_ratio_lead.SetTicks(1, 1)
pad_ratio_lead.SetTopMargin(0.02)  # Reduce top margin to bring plots closer
pad_ratio_lead.SetBottomMargin(0.3)
pad_ratio_lead.Draw()
pad_ratio_lead.cd()

# Calculate and draw ratio plot for leading photon
hist_ratio_lead = hist_data_lead.Clone("hist_ratio_lead")
hist_ratio_lead.Divide(signal_hist_lead)
hist_ratio_lead.SetMarkerStyle(20)
hist_ratio_lead.SetMarkerSize(1.2)
hist_ratio_lead.SetMarkerColor(ROOT.kBlack)
hist_ratio_lead.SetStats(0)
hist_ratio_lead.GetXaxis().SetTitle("Leading Photon P_{T} [GeV]")
hist_ratio_lead.GetYaxis().SetTitle("Data / Signal")
hist_ratio_lead.GetYaxis().SetTitleOffset(0.5)
hist_ratio_lead.GetYaxis().SetTitleSize(0.1)
hist_ratio_lead.GetYaxis().SetLabelSize(0.1)
hist_ratio_lead.GetXaxis().SetTitleSize(0.1)
hist_ratio_lead.GetXaxis().SetLabelSize(0.1)
hist_ratio_lead.GetYaxis().SetNdivisions(505)
hist_ratio_lead.GetYaxis().SetRangeUser(-50, 50)
hist_ratio_lead.GetXaxis().SetTickSize(0.08)
hist_ratio_lead.GetYaxis().SetTickSize(0.05)
hist_ratio_lead.Draw("E1")

# Save the canvas for leading photon as PDF
canvas_lead.SaveAs("/afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Leading_Photon_pt_with_Background_and_Signal_PhotonID.pdf")

# Create a larger canvas for subleading photon
canvas_sublead = ROOT.TCanvas("canvas_sublead", "Subleading Photon Pt", 1000, 1000)
canvas_sublead.SetTicks(1, 1)

# Create pad for main plot for subleading photon
pad_main_sublead = ROOT.TPad("pad_main_sublead", "Main Plot", 0.0, 0.3, 1.0, 1.0)
pad_main_sublead.SetTicks(1, 1)
pad_main_sublead.SetBottomMargin(0.02)  # Reduce bottom margin to hide X-axis numbers
pad_main_sublead.Draw()
pad_main_sublead.cd()

# Draw sublead pt plot
stack_sublead = ROOT.THStack("stack_sublead", "")

# Create data histogram for sublead pt and fill it first
hist_data_sublead = ROOT.TH1F("hist_data_sublead", "", 30, 0, 300)
for sublead_pt in data_sublead_pts:
    hist_data_sublead.Fill(sublead_pt)

hist_data_sublead.SetMarkerStyle(20)
hist_data_sublead.SetMarkerSize(1.2)
hist_data_sublead.SetMarkerColor(ROOT.kBlack)
hist_data_sublead.SetStats(0)  # Remove statistics box
hist_data_sublead.Draw("E1")

for bg_name, hist in background_hists_sublead.items():
    hist.SetStats(0)  # Remove statistics box
    stack_sublead.Add(hist)
stack_sublead.Draw("SAME HIST")

# Set y-axis title and adjust size
stack_sublead.GetYaxis().SetTitle("Events")
stack_sublead.GetYaxis().SetTitleOffset(1.4)
stack_sublead.GetYaxis().SetTitleSize(0.05)  # Increase title size
stack_sublead.GetYaxis().SetLabelSize(0.03)

# Set ticks on all sides with smaller size
stack_sublead.GetXaxis().SetTickSize(0.02)
stack_sublead.GetYaxis().SetTickSize(0.02)

stack_sublead.SetMaximum(50000)  # Set y-axis maximum to 50,000
signal_hist_sublead.SetStats(0)  # Remove statistics box
signal_hist_sublead.Draw("SAME HIST")

# Add legend for sublead pt
legend_sublead = ROOT.TLegend(0.45, 0.55, 0.87, 0.85)
legend_sublead.SetNColumns(2)  # Set number of columns for better layout
legend_sublead.SetBorderSize(1)  # Set border size of the legend box
legend_sublead.SetFillStyle(0)  # Transparent background for the legend box
legend_sublead.SetTextSize(0.02)  # Text size
legend_sublead.AddEntry(hist_data_sublead, "Data", "lep")
legend_sublead.AddEntry(background_hists_sublead["GGJets"], "#gamma#gamma + Jets", "f")
legend_sublead.AddEntry(background_hists_sublead["GJetPt20To40"], "#gamma + Jets (20 < p_{T} < 40)", "f")
legend_sublead.AddEntry(background_hists_sublead["GJetPt40"], "#gamma + Jets (p_{T} > 40)", "f")
legend_sublead.AddEntry(background_hists_sublead["GluGluHToGG"], "(GluGluHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(background_hists_sublead["ttHToGG"], "(ttHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(background_hists_sublead["VBFHToGG"], "(VBFHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(background_hists_sublead["VHToGG"], "(VHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(signal_hist_sublead, "Signal(GluGluToHH)x10", "l")
legend_sublead.Draw()

# Add CMS text for sublead pt
cms_label_sublead = ROOT.TLatex()
cms_label_sublead.SetNDC()
cms_label_sublead.SetTextFont(61)
cms_label_sublead.SetTextSize(0.04)
cms_label_sublead.DrawLatex(0.1, 0.91, "CMS")

# Add "Work in Progress" text for sublead pt
work_label_sublead = ROOT.TLatex()
work_label_sublead.SetNDC()
work_label_sublead.SetTextFont(52)
work_label_sublead.SetTextSize(0.03)
work_label_sublead.DrawLatex(0.17, 0.91, "Work in Progress")

# Add energy information for sublead pt
energy_label_sublead = ROOT.TLatex()
energy_label_sublead.SetNDC()
energy_label_sublead.SetTextFont(42)
energy_label_sublead.SetTextSize(0.03)
energy_label_sublead.DrawLatex(0.81, 0.91, "(13.6 TeV)")

# Create pad for ratio plot for subleading photon
canvas_sublead.cd()
pad_ratio_sublead = ROOT.TPad("pad_ratio_sublead", "Ratio Plot", 0.0, 0.0, 1.0, 0.3)
pad_ratio_sublead.SetTicks(1, 1)
pad_ratio_sublead.SetTopMargin(0.02)  # Reduce top margin to bring plots closer
pad_ratio_sublead.SetBottomMargin(0.3)
pad_ratio_sublead.Draw()
pad_ratio_sublead.cd()

# Calculate and draw ratio plot for subleading photon
hist_ratio_sublead = hist_data_sublead.Clone("hist_ratio_sublead")
hist_ratio_sublead.Divide(signal_hist_sublead)
hist_ratio_sublead.SetMarkerStyle(20)
hist_ratio_sublead.SetMarkerSize(1.2)
hist_ratio_sublead.SetMarkerColor(ROOT.kBlack)
hist_ratio_sublead.SetStats(0)
hist_ratio_sublead.GetXaxis().SetTitle("Subleading Photon P_{T} [GeV]")
hist_ratio_sublead.GetYaxis().SetTitle("Data / Signal")
hist_ratio_sublead.GetYaxis().SetTitleOffset(0.5)
hist_ratio_sublead.GetYaxis().SetTitleSize(0.1)
hist_ratio_sublead.GetYaxis().SetLabelSize(0.1)
hist_ratio_sublead.GetXaxis().SetTitleSize(0.1)
hist_ratio_sublead.GetXaxis().SetLabelSize(0.1)
hist_ratio_sublead.GetYaxis().SetNdivisions(505)
hist_ratio_sublead.GetYaxis().SetRangeUser(-50,50)
hist_ratio_sublead.GetXaxis().SetTickSize(0.08)
hist_ratio_sublead.GetYaxis().SetTickSize(0.05)
hist_ratio_sublead.Draw("E1")

# Save the canvas for subleading photon as PDF
canvas_sublead.SaveAs("/afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Subleading_Photon_pt_with_Background_and_Signal_PhtotonID.pdf")


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_lead
Warning in <TROOT::Append>: Replacing existing TH1: hist_data_lead (Potential memory leak).
Info in <TCanvas::Print>: pdf file /afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Leading_Photon_pt_with_Background_and_Signal_PhotonID.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_sublead
Warning in <TROOT::Append>: Replacing existing TH1: hist_data_sublead (Potential memory leak).
Info in <TCanvas::Print>: pdf file /afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Subleading_Photon_pt_with_Background_and_Signal_PhtotonID.pdf has been created


## Selection for energy 
* 0-100 GeV
* 100- 150 GeV
* 150 - 300 GeV

### 0-100GeV
Applying leading photon for >30GeV and subleading photon for >20GeV


In [37]:
# Create canvases
canvas_lead = ROOT.TCanvas("canvas_lead", "Leading Photon Pt", 800, 800)
canvas_lead.SetTicks(1, 1)

# Create pad for main plot for leading photon
pad_main_lead = ROOT.TPad("pad_main_lead", "Main Plot", 0.0, 0.3, 1.0, 1.0)
pad_main_lead.SetTicks(1, 1)
pad_main_lead.SetBottomMargin(0.02)  # Reduce bottom margin to hide X-axis numbers
pad_main_lead.Draw()
pad_main_lead.cd()

# Draw lead pt plot
stack_lead = ROOT.THStack("stack_lead", "")

# Create data histogram for lead pt and fill it first
hist_data_lead = ROOT.TH1F("hist_data_lead", "", 30, 30, 100)
for lead_pt in data_lead_pts:
    hist_data_lead.Fill(lead_pt)

hist_data_lead.SetMarkerStyle(20)
hist_data_lead.SetMarkerSize(1.2)
hist_data_lead.SetMarkerColor(ROOT.kBlack)
hist_data_lead.SetStats(0)  # Remove statistics box
hist_data_lead.Draw("E1")

for bg_name, hist in background_hists_lead.items():
    hist.SetStats(0)  # Remove statistics box
    stack_lead.Add(hist)
stack_lead.Draw("SAME HIST")

# Set y-axis title and adjust size
stack_lead.GetYaxis().SetTitle("Events")
stack_lead.GetYaxis().SetTitleOffset(1.4)
stack_lead.GetYaxis().SetTitleSize(0.05)  # Increase title size
stack_lead.GetYaxis().SetLabelSize(0.03)

# Set ticks on all sides with smaller size
stack_lead.GetXaxis().SetTickSize(0.02)
stack_lead.GetYaxis().SetTickSize(0.02)

stack_lead.SetMaximum(50000)  # Set y-axis maximum to 50,000
signal_hist_lead.SetStats(0)  # Remove statistics box
signal_hist_lead.Draw("SAME HIST")

# Add legend for lead pt
legend_lead = ROOT.TLegend(0.55, 0.7, 0.85, 0.9)  # Adjust position as needed
legend_lead.SetNColumns(2)  # Set number of columns for better layout
legend_lead.SetBorderSize(1)  # Set border size of the legend box
legend_lead.SetFillStyle(0)  # Transparent background for the legend box
legend_lead.SetTextSize(0.02)  # Text size
legend_lead.AddEntry(hist_data_lead, "Data", "lep")
legend_lead.AddEntry(background_hists_lead["GGJets"], "#gamma#gamma + Jets", "f")
legend_lead.AddEntry(background_hists_lead["GJetPt20To40"], "#gamma + Jets (20 < p_{T} < 40)", "f")
legend_lead.AddEntry(background_hists_lead["GJetPt40"], "#gamma + Jets (p_{T} > 40)", "f")
legend_lead.AddEntry(background_hists_lead["GluGluHToGG"], "(GluGluHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(background_hists_lead["ttHToGG"], "(ttHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(background_hists_lead["VBFHToGG"], "(VBFHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(background_hists_lead["VHToGG"], "(VHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(signal_hist_lead, "Signal(GluGluToHH)x10", "l")
legend_lead.Draw()

# Add CMS text for lead pt
cms_label_lead = ROOT.TLatex()
cms_label_lead.SetNDC()
cms_label_lead.SetTextFont(61)
cms_label_lead.SetTextSize(0.04)
cms_label_lead.DrawLatex(0.1, 0.91, "CMS")

# Add "Work in Progress" text for lead pt
work_label_lead = ROOT.TLatex()
work_label_lead.SetNDC()
work_label_lead.SetTextFont(52)
work_label_lead.SetTextSize(0.03)
work_label_lead.DrawLatex(0.17, 0.91, "Work in Progress")

# Add energy information for lead pt
energy_label_lead = ROOT.TLatex()
energy_label_lead.SetNDC()
energy_label_lead.SetTextFont(42)
energy_label_lead.SetTextSize(0.03)
energy_label_lead.DrawLatex(0.81, 0.91, "(13.6 TeV)")

# Create pad for ratio plot for leading photon
canvas_lead.cd()
pad_ratio_lead = ROOT.TPad("pad_ratio_lead", "Ratio Plot", 0.0, 0.0, 1.0, 0.3)
pad_ratio_lead.SetTicks(1, 1)
pad_ratio_lead.SetTopMargin(0.02)  # Reduce top margin to bring plots closer
pad_ratio_lead.SetBottomMargin(0.3)
pad_ratio_lead.Draw()
pad_ratio_lead.cd()

# Calculate and draw ratio plot for leading photon
hist_ratio_lead = hist_data_lead.Clone("hist_ratio_lead")
hist_ratio_lead.Divide(signal_hist_lead)
hist_ratio_lead.SetMarkerStyle(20)
hist_ratio_lead.SetMarkerSize(1.2)
hist_ratio_lead.SetMarkerColor(ROOT.kBlack)
hist_ratio_lead.SetStats(0)
hist_ratio_lead.GetXaxis().SetTitle("Leading Photon P_{T} [GeV]")
hist_ratio_lead.GetYaxis().SetTitle("Data / Signal")
hist_ratio_lead.GetYaxis().SetTitleOffset(0.5)
hist_ratio_lead.GetYaxis().SetTitleSize(0.1)
hist_ratio_lead.GetYaxis().SetLabelSize(0.1)
hist_ratio_lead.GetXaxis().SetTitleSize(0.1)
hist_ratio_lead.GetXaxis().SetLabelSize(0.1)
hist_ratio_lead.GetYaxis().SetNdivisions(505)
hist_ratio_lead.GetYaxis().SetRangeUser(-50, 50)
hist_ratio_lead.GetXaxis().SetTickSize(0.08)
hist_ratio_lead.GetYaxis().SetTickSize(0.05)
hist_ratio_lead.Draw("E1")

# Save the canvas for leading photon as PDF
canvas_lead.SaveAs("/afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Leading_Photon_pt_with_Background_and_Signal_PhotonID_0-100.pdf")

# Create a larger canvas for subleading photon
canvas_sublead = ROOT.TCanvas("canvas_sublead", "Subleading Photon Pt", 1000, 1000)
canvas_sublead.SetTicks(1, 1)

# Create pad for main plot for subleading photon
pad_main_sublead = ROOT.TPad("pad_main_sublead", "Main Plot", 0.0, 0.3, 1.0, 1.0)
pad_main_sublead.SetTicks(1, 1)
pad_main_sublead.SetBottomMargin(0.02)  # Reduce bottom margin to hide X-axis numbers
pad_main_sublead.Draw()
pad_main_sublead.cd()

# Draw sublead pt plot
stack_sublead = ROOT.THStack("stack_sublead", "")

# Create data histogram for sublead pt and fill it first
hist_data_sublead = ROOT.TH1F("hist_data_sublead", "", 30, 20, 100)
for sublead_pt in data_sublead_pts:
    hist_data_sublead.Fill(sublead_pt)

hist_data_sublead.SetMarkerStyle(20)
hist_data_sublead.SetMarkerSize(1.2)
hist_data_sublead.SetMarkerColor(ROOT.kBlack)
hist_data_sublead.SetStats(0)  # Remove statistics box
hist_data_sublead.Draw("E1")

for bg_name, hist in background_hists_sublead.items():
    hist.SetStats(0)  # Remove statistics box
    stack_sublead.Add(hist)
stack_sublead.Draw("SAME HIST")

# Set y-axis title and adjust size
stack_sublead.GetYaxis().SetTitle("Events")
stack_sublead.GetYaxis().SetTitleOffset(1.4)
stack_sublead.GetYaxis().SetTitleSize(0.05)  # Increase title size
stack_sublead.GetYaxis().SetLabelSize(0.03)

# Set ticks on all sides with smaller size
stack_sublead.GetXaxis().SetTickSize(0.02)
stack_sublead.GetYaxis().SetTickSize(0.02)

stack_sublead.SetMaximum(50000)  # Set y-axis maximum to 50,000
signal_hist_sublead.SetStats(0)  # Remove statistics box
signal_hist_sublead.Draw("SAME HIST")

# Add legend for subleading pt
legend_sublead = ROOT.TLegend(0.55, 0.7, 0.85, 0.9)  # Adjust position as needed
legend_sublead.SetNColumns(2)  # Set number of columns for better layout
legend_sublead.SetBorderSize(1)  # Set border size of the legend box
legend_sublead.SetFillStyle(0)  # Transparent background for the legend box
legend_sublead.SetTextSize(0.02)  # Text size
legend_sublead.AddEntry(hist_data_sublead, "Data", "lep")
legend_sublead.AddEntry(background_hists_sublead["GGJets"], "#gamma#gamma + Jets", "f")
legend_sublead.AddEntry(background_hists_sublead["GJetPt20To40"], "#gamma + Jets (20 < p_{T} < 40)", "f")
legend_sublead.AddEntry(background_hists_sublead["GJetPt40"], "#gamma + Jets (p_{T} > 40)", "f")
legend_sublead.AddEntry(background_hists_sublead["GluGluHToGG"], "(GluGluHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(background_hists_sublead["ttHToGG"], "(ttHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(background_hists_sublead["VBFHToGG"], "(VBFHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(background_hists_sublead["VHToGG"], "(VHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(signal_hist_sublead, "Signal(GluGluToHH)x10", "l")
legend_sublead.Draw()

# Add CMS text for subleading pt
cms_label_sublead = ROOT.TLatex()
cms_label_sublead.SetNDC()
cms_label_sublead.SetTextFont(61)
cms_label_sublead.SetTextSize(0.04)
cms_label_sublead.DrawLatex(0.1, 0.91, "CMS")

# Add "Work in Progress" text for subleading pt
work_label_sublead = ROOT.TLatex()
work_label_sublead.SetNDC()
work_label_sublead.SetTextFont(52)
work_label_sublead.SetTextSize(0.03)
work_label_sublead.DrawLatex(0.17, 0.91, "Work in Progress")

# Add energy information for subleading pt
energy_label_sublead = ROOT.TLatex()
energy_label_sublead.SetNDC()
energy_label_sublead.SetTextFont(42)
energy_label_sublead.SetTextSize(0.03)
energy_label_sublead.DrawLatex(0.81, 0.91, "(13.6 TeV)")

# Create pad for ratio plot for subleading photon
canvas_sublead.cd()
pad_ratio_sublead = ROOT.TPad("pad_ratio_sublead", "Ratio Plot", 0.0, 0.0, 1.0, 0.3)
pad_ratio_sublead.SetTicks(1, 1)
pad_ratio_sublead.SetTopMargin(0.02)  # Reduce top margin to bring plots closer
pad_ratio_sublead.SetBottomMargin(0.3)
pad_ratio_sublead.Draw()
pad_ratio_sublead.cd()

# Calculate and draw ratio plot for subleading photon
hist_ratio_sublead = hist_data_sublead.Clone("hist_ratio_sublead")
hist_ratio_sublead.Divide(signal_hist_sublead)
hist_ratio_sublead.SetMarkerStyle(20)
hist_ratio_sublead.SetMarkerSize(1.2)
hist_ratio_sublead.SetMarkerColor(ROOT.kBlack)
hist_ratio_sublead.SetStats(0)
hist_ratio_sublead.GetXaxis().SetTitle("Subleading Photon P_{T} [GeV]")
hist_ratio_sublead.GetYaxis().SetTitle("Data / Signal")
hist_ratio_sublead.GetYaxis().SetTitleOffset(0.5)
hist_ratio_sublead.GetYaxis().SetTitleSize(0.1)
hist_ratio_sublead.GetYaxis().SetLabelSize(0.1)
hist_ratio_sublead.GetXaxis().SetTitleSize(0.1)
hist_ratio_sublead.GetXaxis().SetLabelSize(0.1)
hist_ratio_sublead.GetYaxis().SetNdivisions(505)
hist_ratio_sublead.GetYaxis().SetRangeUser(-50, 50)
hist_ratio_sublead.GetXaxis().SetTickSize(0.08)
hist_ratio_sublead.GetYaxis().SetTickSize(0.05)
hist_ratio_sublead.Draw("E1")

# Save the canvas for subleading photon as PDF
canvas_sublead.SaveAs("/afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Subleading_Photon_pt_with_Background_and_Signal_PhotonID_0-100.pdf")


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_lead
Warning in <TROOT::Append>: Replacing existing TH1: hist_data_lead (Potential memory leak).
Warning in <TH1F::Divide>: Dividing histograms with different axis limits
Info in <TCanvas::Print>: pdf file /afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Leading_Photon_pt_with_Background_and_Signal_PhotonID_0-100.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_sublead
Warning in <TROOT::Append>: Replacing existing TH1: hist_data_sublead (Potential memory leak).
Error in <TH1F::Divide>: Cannot divide histograms with different number of bins
Info in <TCanvas::Print>: pdf file /afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Subleading_Photon_pt_with_Background_and_Signal_PhotonID_0-100.pdf has been created


### 100-150

In [42]:
# Create canvases
canvas_lead = ROOT.TCanvas("canvas_lead", "Leading Photon Pt", 800, 800)
canvas_lead.SetTicks(1, 1)

# Create pad for main plot for leading photon
pad_main_lead = ROOT.TPad("pad_main_lead", "Main Plot", 0.0, 0.3, 1.0, 1.0)
pad_main_lead.SetTicks(1, 1)
pad_main_lead.SetBottomMargin(0.02)  # Reduce bottom margin to hide X-axis numbers
pad_main_lead.Draw()
pad_main_lead.cd()

# Draw lead pt plot
stack_lead = ROOT.THStack("stack_lead", "")

# Create data histogram for lead pt and fill it first
hist_data_lead = ROOT.TH1F("hist_data_lead", "", 30, 100, 150)
for lead_pt in data_lead_pts:
    hist_data_lead.Fill(lead_pt)

hist_data_lead.SetMarkerStyle(20)
hist_data_lead.SetMarkerSize(1.2)
hist_data_lead.SetMarkerColor(ROOT.kBlack)
hist_data_lead.SetStats(0)  # Remove statistics box
hist_data_lead.Draw("E1")

for bg_name, hist in background_hists_lead.items():
    hist.SetStats(0)  # Remove statistics box
    stack_lead.Add(hist)
stack_lead.Draw("SAME HIST")

# Set y-axis title and adjust size
stack_lead.GetYaxis().SetTitle("Events")
stack_lead.GetYaxis().SetTitleOffset(1.4)
stack_lead.GetYaxis().SetTitleSize(0.05)  # Increase title size
stack_lead.GetYaxis().SetLabelSize(0.03)

# Set ticks on all sides with smaller size
stack_lead.GetXaxis().SetTickSize(0.02)
stack_lead.GetYaxis().SetTickSize(0.02)

stack_lead.SetMaximum(50000)  # Set y-axis maximum to 50,000
signal_hist_lead.SetStats(0)  # Remove statistics box
signal_hist_lead.Draw("SAME HIST")

# Add legend for lead pt
legend_lead = ROOT.TLegend(0.55, 0.7, 0.85, 0.9)  # Adjust position as needed
legend_lead.SetNColumns(2)  # Set number of columns for better layout
legend_lead.SetBorderSize(1)  # Set border size of the legend box
legend_lead.SetFillStyle(0)  # Transparent background for the legend box
legend_lead.SetTextSize(0.02)  # Text size
legend_lead.AddEntry(hist_data_lead, "Data", "lep")
legend_lead.AddEntry(background_hists_lead["GGJets"], "#gamma#gamma + Jets", "f")
legend_lead.AddEntry(background_hists_lead["GJetPt20To40"], "#gamma + Jets (20 < p_{T} < 40)", "f")
legend_lead.AddEntry(background_hists_lead["GJetPt40"], "#gamma + Jets (p_{T} > 40)", "f")
legend_lead.AddEntry(background_hists_lead["GluGluHToGG"], "(GluGluHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(background_hists_lead["ttHToGG"], "(ttHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(background_hists_lead["VBFHToGG"], "(VBFHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(background_hists_lead["VHToGG"], "(VHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(signal_hist_lead, "Signal(GluGluToHH)x10", "l")
legend_lead.Draw()

# Add CMS text for lead pt
cms_label_lead = ROOT.TLatex()
cms_label_lead.SetNDC()
cms_label_lead.SetTextFont(61)
cms_label_lead.SetTextSize(0.04)
cms_label_lead.DrawLatex(0.1, 0.91, "CMS")

# Add "Work in Progress" text for lead pt
work_label_lead = ROOT.TLatex()
work_label_lead.SetNDC()
work_label_lead.SetTextFont(52)
work_label_lead.SetTextSize(0.03)
work_label_lead.DrawLatex(0.17, 0.91, "Work in Progress")

# Add energy information for lead pt
energy_label_lead = ROOT.TLatex()
energy_label_lead.SetNDC()
energy_label_lead.SetTextFont(42)
energy_label_lead.SetTextSize(0.03)
energy_label_lead.DrawLatex(0.81, 0.91, "(13.6 TeV)")

# Create pad for ratio plot for leading photon
canvas_lead.cd()
pad_ratio_lead = ROOT.TPad("pad_ratio_lead", "Ratio Plot", 0.0, 0.0, 1.0, 0.3)
pad_ratio_lead.SetTicks(1, 1)
pad_ratio_lead.SetTopMargin(0.02)  # Reduce top margin to bring plots closer
pad_ratio_lead.SetBottomMargin(0.3)
pad_ratio_lead.Draw()
pad_ratio_lead.cd()

# Calculate and draw ratio plot for leading photon
hist_ratio_lead = hist_data_lead.Clone("hist_ratio_lead")
hist_ratio_lead.Divide(signal_hist_lead)
hist_ratio_lead.SetMarkerStyle(20)
hist_ratio_lead.SetMarkerSize(1.2)
hist_ratio_lead.SetMarkerColor(ROOT.kBlack)
hist_ratio_lead.SetStats(0)
hist_ratio_lead.GetXaxis().SetTitle("Leading Photon P_{T} [GeV]")
hist_ratio_lead.GetYaxis().SetTitle("Data / Signal")
hist_ratio_lead.GetYaxis().SetTitleOffset(0.5)
hist_ratio_lead.GetYaxis().SetTitleSize(0.1)
hist_ratio_lead.GetYaxis().SetLabelSize(0.1)
hist_ratio_lead.GetXaxis().SetTitleSize(0.1)
hist_ratio_lead.GetXaxis().SetLabelSize(0.1)
hist_ratio_lead.GetYaxis().SetNdivisions(505)
hist_ratio_lead.GetYaxis().SetRangeUser(-50, 50)
hist_ratio_lead.GetXaxis().SetTickSize(0.08)
hist_ratio_lead.GetYaxis().SetTickSize(0.05)
hist_ratio_lead.Draw("E1")

# Save the canvas for leading photon as PDF
canvas_lead.SaveAs("/afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Leading_Photon_pt_with_Background_and_Signal_PhotonID_100-150.pdf")

# Create a larger canvas for subleading photon
canvas_sublead = ROOT.TCanvas("canvas_sublead", "Subleading Photon Pt", 1000, 1000)
canvas_sublead.SetTicks(1, 1)

# Create pad for main plot for subleading photon
pad_main_sublead = ROOT.TPad("pad_main_sublead", "Main Plot", 0.0, 0.3, 1.0, 1.0)
pad_main_sublead.SetTicks(1, 1)
pad_main_sublead.SetBottomMargin(0.02)  # Reduce bottom margin to hide X-axis numbers
pad_main_sublead.Draw()
pad_main_sublead.cd()

# Draw sublead pt plot
stack_sublead = ROOT.THStack("stack_sublead", "")

# Create data histogram for sublead pt and fill it first
hist_data_sublead = ROOT.TH1F("hist_data_sublead", "", 30, 100, 150)
for sublead_pt in data_sublead_pts:
    hist_data_sublead.Fill(sublead_pt)

hist_data_sublead.SetMarkerStyle(20)
hist_data_sublead.SetMarkerSize(1.2)
hist_data_sublead.SetMarkerColor(ROOT.kBlack)
hist_data_sublead.SetStats(0)  # Remove statistics box
hist_data_sublead.Draw("E1")

for bg_name, hist in background_hists_sublead.items():
    hist.SetStats(0)  # Remove statistics box
    stack_sublead.Add(hist)
stack_sublead.Draw("SAME HIST")

# Set y-axis title and adjust size
stack_sublead.GetYaxis().SetTitle("Events")
stack_sublead.GetYaxis().SetTitleOffset(1.4)
stack_sublead.GetYaxis().SetTitleSize(0.05)  # Increase title size
stack_sublead.GetYaxis().SetLabelSize(0.03)

# Set ticks on all sides with smaller size
stack_sublead.GetXaxis().SetTickSize(0.02)
stack_sublead.GetYaxis().SetTickSize(0.02)

stack_sublead.SetMaximum(50000)  # Set y-axis maximum to 50,000
signal_hist_sublead.SetStats(0)  # Remove statistics box
signal_hist_sublead.Draw("SAME HIST")

# Add legend for subleading pt
legend_sublead = ROOT.TLegend(0.55, 0.7, 0.85, 0.9)  # Adjust position as needed
legend_sublead.SetNColumns(2)  # Set number of columns for better layout
legend_sublead.SetBorderSize(1)  # Set border size of the legend box
legend_sublead.SetFillStyle(0)  # Transparent background for the legend box
legend_sublead.SetTextSize(0.02)  # Text size
legend_sublead.AddEntry(hist_data_sublead, "Data", "lep")
legend_sublead.AddEntry(background_hists_sublead["GGJets"], "#gamma#gamma + Jets", "f")
legend_sublead.AddEntry(background_hists_sublead["GJetPt20To40"], "#gamma + Jets (20 < p_{T} < 40)", "f")
legend_sublead.AddEntry(background_hists_sublead["GJetPt40"], "#gamma + Jets (p_{T} > 40)", "f")
legend_sublead.AddEntry(background_hists_sublead["GluGluHToGG"], "(GluGluHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(background_hists_sublead["ttHToGG"], "(ttHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(background_hists_sublead["VBFHToGG"], "(VBFHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(background_hists_sublead["VHToGG"], "(VHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(signal_hist_sublead, "Signal(GluGluToHH)x10", "l")
legend_sublead.Draw()

# Add CMS text for subleading pt
cms_label_sublead = ROOT.TLatex()
cms_label_sublead.SetNDC()
cms_label_sublead.SetTextFont(61)
cms_label_sublead.SetTextSize(0.04)
cms_label_sublead.DrawLatex(0.1, 0.91, "CMS")

# Add "Work in Progress" text for subleading pt
work_label_sublead = ROOT.TLatex()
work_label_sublead.SetNDC()
work_label_sublead.SetTextFont(52)
work_label_sublead.SetTextSize(0.03)
work_label_sublead.DrawLatex(0.17, 0.91, "Work in Progress")

# Add energy information for subleading pt
energy_label_sublead = ROOT.TLatex()
energy_label_sublead.SetNDC()
energy_label_sublead.SetTextFont(42)
energy_label_sublead.SetTextSize(0.03)
energy_label_sublead.DrawLatex(0.81, 0.91, "(13.6 TeV)")

# Create pad for ratio plot for subleading photon
canvas_sublead.cd()
pad_ratio_sublead = ROOT.TPad("pad_ratio_sublead", "Ratio Plot", 0.0, 0.0, 1.0, 0.3)
pad_ratio_sublead.SetTicks(1, 1)
pad_ratio_sublead.SetTopMargin(0.02)  # Reduce top margin to bring plots closer
pad_ratio_sublead.SetBottomMargin(0.3)
pad_ratio_sublead.Draw()
pad_ratio_sublead.cd()

# Calculate and draw ratio plot for subleading photon
hist_ratio_sublead = hist_data_sublead.Clone("hist_ratio_sublead")
hist_ratio_sublead.Divide(signal_hist_sublead)
hist_ratio_sublead.SetMarkerStyle(20)
hist_ratio_sublead.SetMarkerSize(1.2)
hist_ratio_sublead.SetMarkerColor(ROOT.kBlack)
hist_ratio_sublead.SetStats(0)
hist_ratio_sublead.GetXaxis().SetTitle("Subleading Photon P_{T} [GeV]")
hist_ratio_sublead.GetYaxis().SetTitle("Data / Signal")
hist_ratio_sublead.GetYaxis().SetTitleOffset(0.5)
hist_ratio_sublead.GetYaxis().SetTitleSize(0.1)
hist_ratio_sublead.GetYaxis().SetLabelSize(0.1)
hist_ratio_sublead.GetXaxis().SetTitleSize(0.1)
hist_ratio_sublead.GetXaxis().SetLabelSize(0.1)
hist_ratio_sublead.GetYaxis().SetNdivisions(505)
hist_ratio_sublead.GetYaxis().SetRangeUser(-50, 50)
hist_ratio_sublead.GetXaxis().SetTickSize(0.08)
hist_ratio_sublead.GetYaxis().SetTickSize(0.05)
hist_ratio_sublead.Draw("E1")

# Save the canvas for subleading photon as PDF
canvas_sublead.SaveAs("/afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Subleading_Photon_pt_with_Background_and_Signal_PhotonID_100-150.pdf")


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_lead
Warning in <TROOT::Append>: Replacing existing TH1: hist_data_lead (Potential memory leak).
Info in <TCanvas::Print>: pdf file /afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Leading_Photon_pt_with_Background_and_Signal_PhotonID_100-150.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_sublead
Warning in <TROOT::Append>: Replacing existing TH1: hist_data_sublead (Potential memory leak).
Error in <TH1F::Divide>: Cannot divide histograms with different number of bins
Info in <TCanvas::Print>: pdf file /afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Subleading_Photon_pt_with_Background_and_Signal_PhotonID_100-150.pdf has been created


# 150 - 200 GeV


In [51]:
# Create canvases
canvas_lead = ROOT.TCanvas("canvas_lead", "Leading Photon Pt", 800, 800)
canvas_lead.SetTicks(1, 1)

# Create pad for main plot for leading photon
pad_main_lead = ROOT.TPad("pad_main_lead", "Main Plot", 0.0, 0.3, 1.0, 1.0)
pad_main_lead.SetTicks(1, 1)
pad_main_lead.SetBottomMargin(0.02)  # Reduce bottom margin to hide X-axis numbers
pad_main_lead.Draw()
pad_main_lead.cd()

# Draw lead pt plot
stack_lead = ROOT.THStack("stack_lead", "")

# Create data histogram for lead pt and fill it first
hist_data_lead = ROOT.TH1F("hist_data_lead", "", 30, 150, 200)
for lead_pt in data_lead_pts:
    hist_data_lead.Fill(lead_pt)

hist_data_lead.SetMarkerStyle(20)
hist_data_lead.SetMarkerSize(1.2)
hist_data_lead.SetMarkerColor(ROOT.kBlack)
hist_data_lead.SetStats(0)  # Remove statistics box
hist_data_lead.Draw("E1")

for bg_name, hist in background_hists_lead.items():
    hist.SetStats(0)  # Remove statistics box
    stack_lead.Add(hist)
stack_lead.Draw("SAME HIST")

# Set y-axis title and adjust size
stack_lead.GetYaxis().SetTitle("Events")
stack_lead.GetYaxis().SetTitleOffset(1.4)
stack_lead.GetYaxis().SetTitleSize(0.05)  # Increase title size
stack_lead.GetYaxis().SetLabelSize(0.03)

# Set ticks on all sides with smaller size
stack_lead.GetXaxis().SetTickSize(0.02)
stack_lead.GetYaxis().SetTickSize(0.02)

stack_lead.SetMaximum(50000)  # Set y-axis maximum to 50,000
signal_hist_lead.SetStats(0)  # Remove statistics box
signal_hist_lead.Draw("SAME HIST")

# Add legend for lead pt
legend_lead = ROOT.TLegend(0.55, 0.7, 0.85, 0.9)  # Adjust position as needed
legend_lead.SetNColumns(2)  # Set number of columns for better layout
legend_lead.SetBorderSize(1)  # Set border size of the legend box
legend_lead.SetFillStyle(0)  # Transparent background for the legend box
legend_lead.SetTextSize(0.02)  # Text size
legend_lead.AddEntry(hist_data_lead, "Data", "lep")
legend_lead.AddEntry(background_hists_lead["GGJets"], "#gamma#gamma + Jets", "f")
legend_lead.AddEntry(background_hists_lead["GJetPt20To40"], "#gamma + Jets (20 < p_{T} < 40)", "f")
legend_lead.AddEntry(background_hists_lead["GJetPt40"], "#gamma + Jets (p_{T} > 40)", "f")
legend_lead.AddEntry(background_hists_lead["GluGluHToGG"], "(GluGluHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(background_hists_lead["ttHToGG"], "(ttHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(background_hists_lead["VBFHToGG"], "(VBFHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(background_hists_lead["VHToGG"], "(VHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(signal_hist_lead, "Signal(GluGluToHH)x10", "l")
legend_lead.Draw()

# Add CMS text for lead pt
cms_label_lead = ROOT.TLatex()
cms_label_lead.SetNDC()
cms_label_lead.SetTextFont(61)
cms_label_lead.SetTextSize(0.04)
cms_label_lead.DrawLatex(0.1, 0.91, "CMS")

# Add "Work in Progress" text for lead pt
work_label_lead = ROOT.TLatex()
work_label_lead.SetNDC()
work_label_lead.SetTextFont(52)
work_label_lead.SetTextSize(0.03)
work_label_lead.DrawLatex(0.17, 0.91, "Work in Progress")

# Add energy information for lead pt
energy_label_lead = ROOT.TLatex()
energy_label_lead.SetNDC()
energy_label_lead.SetTextFont(42)
energy_label_lead.SetTextSize(0.03)
energy_label_lead.DrawLatex(0.81, 0.91, "(13.6 TeV)")

# Create pad for ratio plot for leading photon
canvas_lead.cd()
pad_ratio_lead = ROOT.TPad("pad_ratio_lead", "Ratio Plot", 0.0, 0.0, 1.0, 0.3)
pad_ratio_lead.SetTicks(1, 1)
pad_ratio_lead.SetTopMargin(0.02)  # Reduce top margin to bring plots closer
pad_ratio_lead.SetBottomMargin(0.3)
pad_ratio_lead.Draw()
pad_ratio_lead.cd()

# Calculate and draw ratio plot for leading photon
hist_ratio_lead = hist_data_lead.Clone("hist_ratio_lead")
hist_ratio_lead.Divide(signal_hist_lead)
hist_ratio_lead.SetMarkerStyle(20)
hist_ratio_lead.SetMarkerSize(1.2)
hist_ratio_lead.SetMarkerColor(ROOT.kBlack)
hist_ratio_lead.SetStats(0)
hist_ratio_lead.GetXaxis().SetTitle("Leading Photon P_{T} [GeV]")
hist_ratio_lead.GetYaxis().SetTitle("Data / Signal")
hist_ratio_lead.GetYaxis().SetTitleOffset(0.5)
hist_ratio_lead.GetYaxis().SetTitleSize(0.1)
hist_ratio_lead.GetYaxis().SetLabelSize(0.1)
hist_ratio_lead.GetXaxis().SetTitleSize(0.1)
hist_ratio_lead.GetXaxis().SetLabelSize(0.1)
hist_ratio_lead.GetYaxis().SetNdivisions(505)
hist_ratio_lead.GetYaxis().SetRangeUser(-0.5, 3.5)
hist_ratio_lead.GetXaxis().SetTickSize(0.08)
hist_ratio_lead.GetYaxis().SetTickSize(0.05)
hist_ratio_lead.Draw("E1")

# Save the canvas for leading photon as PDF
canvas_lead.SaveAs("/afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Leading_Photon_pt_with_Background_and_Signal_PhotonID_150-200.pdf")

# Create a larger canvas for subleading photon
canvas_sublead = ROOT.TCanvas("canvas_sublead", "Subleading Photon Pt", 1000, 1000)
canvas_sublead.SetTicks(1, 1)

# Create pad for main plot for subleading photon
pad_main_sublead = ROOT.TPad("pad_main_sublead", "Main Plot", 0.0, 0.3, 1.0, 1.0)
pad_main_sublead.SetTicks(1, 1)
pad_main_sublead.SetBottomMargin(0.02)  # Reduce bottom margin to hide X-axis numbers
pad_main_sublead.Draw()
pad_main_sublead.cd()

# Draw sublead pt plot
stack_sublead = ROOT.THStack("stack_sublead", "")

# Create data histogram for sublead pt and fill it first
hist_data_sublead = ROOT.TH1F("hist_data_sublead", "", 30, 150, 200)
for sublead_pt in data_sublead_pts:
    hist_data_sublead.Fill(sublead_pt)

hist_data_sublead.SetMarkerStyle(20)
hist_data_sublead.SetMarkerSize(1.2)
hist_data_sublead.SetMarkerColor(ROOT.kBlack)
hist_data_sublead.SetStats(0)  # Remove statistics box
hist_data_sublead.Draw("E1")

for bg_name, hist in background_hists_sublead.items():
    hist.SetStats(0)  # Remove statistics box
    stack_sublead.Add(hist)
stack_sublead.Draw("SAME HIST")

# Set y-axis title and adjust size
stack_sublead.GetYaxis().SetTitle("Events")
stack_sublead.GetYaxis().SetTitleOffset(1.4)
stack_sublead.GetYaxis().SetTitleSize(0.05)  # Increase title size
stack_sublead.GetYaxis().SetLabelSize(0.03)

# Set ticks on all sides with smaller size
stack_sublead.GetXaxis().SetTickSize(0.02)
stack_sublead.GetYaxis().SetTickSize(0.02)

stack_sublead.SetMaximum(50000)  # Set y-axis maximum to 50,000
signal_hist_sublead.SetStats(0)  # Remove statistics box
signal_hist_sublead.Draw("SAME HIST")

# Add legend for subleading pt
legend_sublead = ROOT.TLegend(0.55, 0.7, 0.85, 0.9)  # Adjust position as needed
legend_sublead.SetNColumns(2)  # Set number of columns for better layout
legend_sublead.SetBorderSize(1)  # Set border size of the legend box
legend_sublead.SetFillStyle(0)  # Transparent background for the legend box
legend_sublead.SetTextSize(0.02)  # Text size
legend_sublead.AddEntry(hist_data_sublead, "Data", "lep")
legend_sublead.AddEntry(background_hists_sublead["GGJets"], "#gamma#gamma + Jets", "f")
legend_sublead.AddEntry(background_hists_sublead["GJetPt20To40"], "#gamma + Jets (20 < p_{T} < 40)", "f")
legend_sublead.AddEntry(background_hists_sublead["GJetPt40"], "#gamma + Jets (p_{T} > 40)", "f")
legend_sublead.AddEntry(background_hists_sublead["GluGluHToGG"], "(GluGluHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(background_hists_sublead["ttHToGG"], "(ttHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(background_hists_sublead["VBFHToGG"], "(VBFHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(background_hists_sublead["VHToGG"], "(VHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(signal_hist_sublead, "Signal(GluGluToHH)x10", "l")
legend_sublead.Draw()

# Add CMS text for subleading pt
cms_label_sublead = ROOT.TLatex()
cms_label_sublead.SetNDC()
cms_label_sublead.SetTextFont(61)
cms_label_sublead.SetTextSize(0.04)
cms_label_sublead.DrawLatex(0.1, 0.91, "CMS")

# Add "Work in Progress" text for subleading pt
work_label_sublead = ROOT.TLatex()
work_label_sublead.SetNDC()
work_label_sublead.SetTextFont(52)
work_label_sublead.SetTextSize(0.03)
work_label_sublead.DrawLatex(0.17, 0.91, "Work in Progress")

# Add energy information for subleading pt
energy_label_sublead = ROOT.TLatex()
energy_label_sublead.SetNDC()
energy_label_sublead.SetTextFont(42)
energy_label_sublead.SetTextSize(0.03)
energy_label_sublead.DrawLatex(0.81, 0.91, "(13.6 TeV)")

# Create pad for ratio plot for subleading photon
canvas_sublead.cd()
pad_ratio_sublead = ROOT.TPad("pad_ratio_sublead", "Ratio Plot", 0.0, 0.0, 1.0, 0.3)
pad_ratio_sublead.SetTicks(1, 1)
pad_ratio_sublead.SetTopMargin(0.02)  # Reduce top margin to bring plots closer
pad_ratio_sublead.SetBottomMargin(0.3)
pad_ratio_sublead.Draw()
pad_ratio_sublead.cd()

# Calculate and draw ratio plot for subleading photon
hist_ratio_sublead = hist_data_sublead.Clone("hist_ratio_sublead")
hist_ratio_sublead.Divide(signal_hist_sublead)
hist_ratio_sublead.SetMarkerStyle(20)
hist_ratio_sublead.SetMarkerSize(1.2)
hist_ratio_sublead.SetMarkerColor(ROOT.kBlack)
hist_ratio_sublead.SetStats(0)
hist_ratio_sublead.GetXaxis().SetTitle("Subleading Photon P_{T} [GeV]")
hist_ratio_sublead.GetYaxis().SetTitle("Data / Signal")
hist_ratio_sublead.GetYaxis().SetTitleOffset(0.5)
hist_ratio_sublead.GetYaxis().SetTitleSize(0.1)
hist_ratio_sublead.GetYaxis().SetLabelSize(0.1)
hist_ratio_sublead.GetXaxis().SetTitleSize(0.1)
hist_ratio_sublead.GetXaxis().SetLabelSize(0.1)
hist_ratio_sublead.GetYaxis().SetNdivisions(505)
hist_ratio_sublead.GetYaxis().SetRangeUser(-0.5, 3.5)
hist_ratio_sublead.GetXaxis().SetTickSize(0.08)
hist_ratio_sublead.GetYaxis().SetTickSize(0.05)
hist_ratio_sublead.Draw("E1")

# Save the canvas for subleading photon as PDF
canvas_sublead.SaveAs("/afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Subleading_Photon_pt_with_Background_and_Signal_PhotonID_150-200.pdf")


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_lead
Warning in <TROOT::Append>: Replacing existing TH1: hist_data_lead (Potential memory leak).
Info in <TCanvas::Print>: pdf file /afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Leading_Photon_pt_with_Background_and_Signal_PhotonID_150-200.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_sublead
Warning in <TROOT::Append>: Replacing existing TH1: hist_data_sublead (Potential memory leak).
Info in <TCanvas::Print>: pdf file /afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Subleading_Photon_pt_with_Background_and_Signal_PhotonID_150-200.pdf has been created


# preselection

In [66]:
# Process data files
data_lead_pts = []
data_sublead_pts = []
for data_file_path in data_file_paths:
    lead_pts, sublead_pts, _, _ = process_file(data_file_path, tree_name)
    # Apply preselection
    lead_pts_selected = [pt for pt in lead_pts if pt > 30]
    sublead_pts_selected = [pt for pt in sublead_pts if pt > 20]
    data_lead_pts.extend(lead_pts_selected)
    data_sublead_pts.extend(sublead_pts_selected)


In [68]:
# Create canvases
canvas_lead = ROOT.TCanvas("canvas_lead", "Leading Photon Pt", 800, 600)
canvas_sublead = ROOT.TCanvas("canvas_sublead", "Subleading Photon Pt", 800, 600)

# Draw lead pt plot
canvas_lead.cd()
stack_lead = ROOT.THStack("stack_lead", "")
for bg_name, hist in background_hists_lead.items():
    stack_lead.Add(hist)
stack_lead.Draw("HIST")
stack_lead.GetXaxis().SetTitle("Leading Photon P_{T} [GeV]")
stack_lead.GetYaxis().SetTitle("Events")
stack_lead.GetYaxis().SetTitleOffset(1.4)
stack_lead.GetYaxis().SetLabelSize(0.03)

# Set ticks on all sides with smaller size
stack_lead.GetXaxis().SetTickSize(0.02)
stack_lead.GetYaxis().SetTickSize(0.02)

max_y_lead = max(stack_lead.GetMaximum(), signal_hist_lead.GetMaximum())
stack_lead.SetMaximum(max_y_lead * 1.5)
signal_hist_lead.Draw("SAME HIST")

# Create data histogram for lead pt
hist_data_lead = ROOT.TH1F("hist_data_lead", "", 30, 0, 300)
for lead_pt in data_lead_pts:
    if lead_pt > 30:
        hist_data_lead.Fill(lead_pt)

hist_data_lead.SetMarkerStyle(20)
hist_data_lead.SetMarkerSize(1.2)
hist_data_lead.SetMarkerColor(ROOT.kBlack)
hist_data_lead.Draw("SAME E1")

# Add legend for lead pt
legend_lead = ROOT.TLegend(0.45, 0.55, 0.87, 0.85)
legend_lead.SetNColumns(2)  # Set number of columns for better layout
legend_lead.SetBorderSize(1)  # Set border size of the legend box
legend_lead.SetFillStyle(0)  # Transparent background for the legend box
legend_lead.SetTextSize(0.02)  # Text size
legend_lead.AddEntry(hist_data_lead, "Data", "lep")
legend_lead.AddEntry(background_hists_lead["GGJets"], "#gamma#gamma + Jets", "f")
legend_lead.AddEntry(background_hists_lead["GJetPt20To40"], "#gamma + Jets (20 < p_{T} < 40)", "f")
legend_lead.AddEntry(background_hists_lead["GJetPt40"], "#gamma + Jets (p_{T} > 40)", "f")
legend_lead.AddEntry(background_hists_lead["GluGluHToGG"], "(GluGluHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(background_hists_lead["ttHToGG"], "(ttHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(background_hists_lead["VBFHToGG"], "(VBFHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_lead.AddEntry(background_hists_lead["VHToGG"], "(VHToGG)Higgs #rightarrow #gamma#gamma", "f")
# for bg_name in background_hists_lead:
#     legend_lead.AddEntry(background_hists_lead[bg_name], bg_name, "f")
legend_lead.AddEntry(signal_hist_lead, "Signal(GluGluToHH)x10", "l")
legend_lead.Draw()

# Add CMS text for lead pt
cms_label_lead = ROOT.TLatex()
cms_label_lead.SetNDC()
cms_label_lead.SetTextFont(61)
cms_label_lead.SetTextSize(0.04)
cms_label_lead.DrawLatex(0.1, 0.91, "CMS")

# Add "Work in Progress" text for lead pt
work_label_lead = ROOT.TLatex()
work_label_lead.SetNDC()
work_label_lead.SetTextFont(52)
work_label_lead.SetTextSize(0.03)
work_label_lead.DrawLatex(0.17, 0.91, "Work in Progress")

# Add energy information for lead pt
energy_label_lead = ROOT.TLatex()
energy_label_lead.SetNDC()
energy_label_lead.SetTextFont(42)
energy_label_lead.SetTextSize(0.03)
energy_label_lead.DrawLatex(0.81, 0.91, "(13.6 TeV)")

# Save the canvas as PDF
canvas_lead.SaveAs("/afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Leading_Photon_Pt_with_Background_and_Signal_preselection.pdf")

# Draw sublead pt plot
canvas_sublead.cd()
stack_sublead = ROOT.THStack("stack_sublead", "")
for bg_name, hist in background_hists_sublead.items():
    stack_sublead.Add(hist)
stack_sublead.Draw("HIST")
stack_sublead.GetXaxis().SetTitle("Subleading Photon P_{T} [GeV]")
stack_sublead.GetYaxis().SetTitle("Events")
stack_sublead.GetYaxis().SetTitleOffset(1.4)
stack_sublead.GetYaxis().SetLabelSize(0.03)

# Set ticks on all sides with smaller size
stack_sublead.GetXaxis().SetTickSize(0.02)
stack_sublead.GetYaxis().SetTickSize(0.02)

max_y_sublead = max(stack_sublead.GetMaximum(), signal_hist_sublead.GetMaximum())
stack_sublead.SetMaximum(max_y_sublead * 1.5)
signal_hist_sublead.Draw("SAME HIST")

# Create data histogram for sublead pt
hist_data_sublead = ROOT.TH1F("hist_data_sublead", "", 30, 0, 300)
for sublead_pt in data_sublead_pts:
    if sublead_pt > 20:
        hist_data_sublead.Fill(sublead_pt)

hist_data_sublead.SetMarkerStyle(20)
hist_data_sublead.SetMarkerSize(1.2)
hist_data_sublead.SetMarkerColor(ROOT.kBlack)
hist_data_sublead.Draw("SAME E1")

# Add legend for sublead pt
legend_sublead = ROOT.TLegend(0.45, 0.55, 0.87, 0.85)
legend_sublead.SetNColumns(2)  # Set number of columns for better layout
legend_sublead.SetBorderSize(1)  # Set border size of the legend box
legend_sublead.SetFillStyle(0)  # Transparent background for the legend box
legend_sublead.SetTextSize(0.02)  # Text size
legend_sublead.AddEntry(hist_data_sublead, "Data", "lep")
legend_sublead.AddEntry(background_hists_lead["GGJets"], "#gamma#gamma + Jets", "f")
legend_sublead.AddEntry(background_hists_lead["GJetPt20To40"], "#gamma + Jets (20 < p_{T} < 40)", "f")
legend_sublead.AddEntry(background_hists_lead["GJetPt40"], "#gamma + Jets (p_{T} > 40)", "f")
legend_sublead.AddEntry(background_hists_lead["GluGluHToGG"], "(GluGluHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(background_hists_lead["ttHToGG"], "(ttHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(background_hists_lead["VBFHToGG"], "(VBFHToGG)Higgs #rightarrow #gamma#gamma", "f")
legend_sublead.AddEntry(background_hists_lead["VHToGG"], "(VHToGG)Higgs #rightarrow #gamma#gamma", "f")
# for bg_name in background_hists_lead:
#     legend_sublead.AddEntry(background_hists_lead[bg_name], bg_name, "f")
legend_sublead.AddEntry(signal_hist_lead, "Signal(GluGluToHH)x10", "l")
legend_sublead.Draw()

# Add CMS text for sublead pt
cms_label_sublead = ROOT.TLatex()
cms_label_sublead.SetNDC()
cms_label_sublead.SetTextFont(61)
cms_label_sublead.SetTextSize(0.04)
cms_label_sublead.DrawLatex(0.1, 0.91, "CMS")

# Add "Work in Progress" text for sublead pt
work_label_sublead = ROOT.TLatex()
work_label_sublead.SetNDC()
work_label_sublead.SetTextFont(52)
work_label_sublead.SetTextSize(0.03)
work_label_sublead.DrawLatex(0.17, 0.91, "Work in Progress")

# Add energy information for sublead pt
energy_label_sublead = ROOT.TLatex()
energy_label_sublead.SetNDC()
energy_label_sublead.SetTextFont(42)
energy_label_sublead.SetTextSize(0.03)
energy_label_sublead.DrawLatex(0.81, 0.91, "(13.6 TeV)")

# Save the canvas as PDF
canvas_sublead.SaveAs("/afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Subleading_Photon_Pt_with_Background_and_Signal_preselection.pdf")


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_lead
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas_sublead
Warning in <TROOT::Append>: Replacing existing TH1: hist_data_lead (Potential memory leak).
Info in <TCanvas::Print>: pdf file /afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Leading_Photon_Pt_with_Background_and_Signal_preselection.pdf has been created
Warning in <TROOT::Append>: Replacing existing TH1: hist_data_sublead (Potential memory leak).
Info in <TCanvas::Print>: pdf file /afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/Subleading_Photon_Pt_with_Background_and_Signal_preselection.pdf has been created
